# S1 Coherence

This notebook contains the necessary code and CWL to run the S1 coherence workflow on ADES.

Create a `.env` in the repository root directory with env vars - see below code cells. Note that you need to pass sentinel hub login credentials to the workflow as inputs. Amend inputs, aoi, etc. as required.


In [1]:
import os

import dotenv
import json

import pyeodh

dotenv.load_dotenv()

client = pyeodh.Client(
    username=os.getenv("ADES_USER"),
    token=os.getenv("ADES_TOKEN"),
)
ades = client.get_ades()

In [2]:
cwl_yaml = r"""cwlVersion: v1.0
$graph:
- class: CommandLineTool
  id: 's1_make_stac'
  inputs:
  - id: 'files'
    doc: FILES
    type:
      type: array
      items: File
    inputBinding: {}
  - id: 'intersects'
    inputBinding:
      prefix: --intersects
    type:
    - 'null'
    - string
  requirements:
  - class: DockerRequirement
    dockerPull: ghcr.io/eo-datahub/user-workflows/s1_make_stac@sha256:d85ca88d722159615d4d3b0c2c9a2db12d71f41f5aa86ab9907f00a347116cff
  - class: InlineJavascriptRequirement
  doc: "None\n"
  baseCommand:
  - python
  - /app/app.py
  outputs:
  - id: 'stac_catalog'
    outputBinding:
      glob: .
    type: Directory
- class: CommandLineTool
  id: 's1_process'
  inputs:
  - id: 'intersects'
    inputBinding:
      prefix: --intersects
    type:
    - 'null'
    - string
  - id: 'pair'
    inputBinding:
      prefix: --pair
    type: File
  - id: 'password'
    inputBinding:
      prefix: --password
    type:
    - 'null'
    - string
  - id: 'username'
    inputBinding:
      prefix: --username
    type:
    - 'null'
    - string
  - id: 'AWS_ACCESS_KEY_ID'
    inputBinding:
      prefix: --aws-access-key-id
    type:
    - 'null'
    - string
  - id: 'AWS_SECRET_ACCESS_KEY'
    inputBinding:
      prefix: --aws-secret-access-key
    type:
    - 'null'
    - string  
  - id: 'AWS_SESSION_TOKEN'
    inputBinding:
      prefix: --aws-session-token
    type:
    - 'null'
    - string
  - id: 'workspace'
    inputBinding:
      prefix: --workspace
    type:
    - 'null'
    - string
  outputs:
  - id: 'coherence'
    outputBinding:
      glob: data/results/coh_vv.tif
    type: File
  requirements:
  - class: DockerRequirement
    dockerPull: ghcr.io/eo-datahub/user-workflows/s1_process@sha256:c076e775b78987a2478b2be528b631c1a3f77e5d5dd8983ed8ccd9c914f6966c
  - class: InlineJavascriptRequirement
  doc: "None\n"
  baseCommand:
  - /usr/local/bin/_entrypoint.sh
  - env
  - HOME=/tmp
  - python
  - /app/app.py
- class: CommandLineTool
  id: 's1_search'
  inputs:
  - id: 'end_datetime'
    inputBinding:
      prefix: --end-datetime
    type:
    - 'null'
    - string
  - id: 'intersects'
    inputBinding:
      prefix: --intersects
    type:
    - 'null'
    - string
  - id: 'password'
    inputBinding:
      prefix: --password
    type:
    - 'null'
    - string
  - id: 'start_datetime'
    inputBinding:
      prefix: --start-datetime
    type:
    - 'null'
    - string
  - id: 'username'
    inputBinding:
      prefix: --username
    type:
    - 'null'
    - string
  outputs:
  - id: 'pairs'
    outputBinding:
      glob: pair_*.geojson
    type:
      items: File
      type: array
  requirements:
  - class: DockerRequirement
    dockerPull: ghcr.io/eo-datahub/user-workflows/s1_search@sha256:408de5a8f70a44f4520e850f0d0987dbc0efbe1f64889e115678a3f5010f3f37
  - class: InlineJavascriptRequirement
  doc: "None\n"
  baseCommand:
  - python
  - /app/app.py
- class: Workflow
  id: 's1-coherence'
  inputs:
  - id: 'intersects'
    label: Intersects
    doc: "a GeoJSON-like json string, which provides a \"type\" member describing
      the type of the geometry and \"coordinates\"  member providing a list of coordinates.
      Will search for images intersecting this geometry.\n"
    default: "{\n  \"type\": \"Polygon\",\n  \"coordinates\": [\n    [\n      [0.08905898091569497,
      52.69722175598818],\n      [0.08905898091569497, 52.15527412683906],\n     \
      \ [0.9565339502005088, 52.15527412683906],\n      [0.9565339502005088, 52.69722175598818],\n\
      \      [0.08905898091569497, 52.69722175598818]\n    ]\n  ]\n}\n"
    type: string
  - id: 'start_datetime'
    label: Start datetime
    doc: Start datetime
    default: '2023-04-01'
    type: string
  - id: 'end_datetime'
    label: End datetime
    doc: End datetime
    default: '2023-06-30'
    type: string
  - id: 'username'
    label: Username
    doc: Username
    type: string
  - id: 'password'
    label: Password
    doc: Password
    type: string
  - id: 'AWS_ACCESS_KEY_ID'
    label: AWS Access Key ID
    doc: AWS Access Key ID
    type: string
  - id: 'AWS_SECRET_ACCESS_KEY'
    label: AWS Secret Access Key
    doc: AWS Secret Access Key
    type: string
  - id: 'AWS_SESSION_TOKEN'
    label: AWS Session Token
    doc: AWS Session Token
    type: string
  - id: workspace_prefix
    label: Workspace prefix
    doc: Workspace prefix
    type: string
  outputs:
  - id: 'stac_output'
    outputSource:
    - 's1_make_stac/stac_catalog'
    type: Directory
  requirements:
  - class: ResourceRequirement
    coresMin: 3
    ramMin: 15000
  - class: ScatterFeatureRequirement
  label: S1 coherence
  doc: Generate Sentinel 1 image pair coherence
  steps:
  - id: 's1_search'
    in:
    - id: 'intersects'
      source: 'intersects'
    - id: 'start_datetime'
      source: 'start_datetime'
    - id: 'end_datetime'
      source: 'end_datetime'
    - id: 'username'
      source: 'username'
    - id: 'password'
      source: 'password'
    out:
    - id: 'pairs'
    run: '#s1_search'
  - id: 's1_process'
    in:
    - id: 'pair'
      source: 's1_search/pairs'
    - id: 'intersects'
      source: 'intersects'
    - id: 'username'
      source: 'username'
    - id: 'password'
      source: 'password'
    - id: 'AWS_ACCESS_KEY_ID'
      source: 'AWS_ACCESS_KEY_ID'
    - id: 'AWS_SECRET_ACCESS_KEY'
      source: 'AWS_SECRET_ACCESS_KEY'
    - id: 'AWS_SESSION_TOKEN'
      source: 'AWS_SESSION_TOKEN'
    - id: 'workspace'
      source: 'workspace_prefix'
    out:
    - id: 'coherence'
    run: '#s1_process'
    scatter:
    - 'pair'
    scatterMethod: dotproduct
  - id: 's1_make_stac'
    in:
    - id: 'intersects'
      source: 'intersects'
    - id: 'files'
      source: 's1_process/coherence'
    out:
    - id: 'stac_catalog'
    run: '#s1_make_stac'
"""

In [3]:
try:
    ades.get_process("s1-coherence").delete()
except Exception:
    print("Process not found, no need to undeploy.")

process = ades.deploy_process(cwl_yaml=cwl_yaml)

In [4]:
intersects = {
    "type": "Polygon",
    "coordinates": [
        [
            [0.08905898091569497, 52.69722175598818],
            [0.08905898091569497, 52.15527412683906],
            [0.9565339502005088, 52.15527412683906],
            [0.9565339502005088, 52.69722175598818],
            [0.08905898091569497, 52.69722175598818],
        ]
    ],
}

j = ades.get_process("s1-coherence").execute(
    {
        "username": os.getenv("SH_USERNAME"),
        "password": os.getenv("SH_PASSWORD"),
        "intersects": json.dumps(intersects),
        "start_datetime": "2023-04-01",
        "end_datetime": "2023-06-30",
        "workspace_prefix": "figi44",
    }
)

In [5]:
j.self_href

'https://staging.eodatahub.org.uk/ades/figi44/ogc-api/jobs/98b0ad2e-059d-11f0-bf8a-eadd75232b79'

In [33]:
j.refresh()
j.status
j.message

'ZOO-Kernel successfully run your service!'

In [36]:
j.get_result_items()[0].assets

{'coh_vv': <Asset href=https://figi44.staging.eodatahub-workspaces.org.uk/files/workspaces-eodhp-staging/processing-results/s1-coherence/cat_eefbd778-04ec-11f0-b747-eadd75232b79/col_eefbd778-04ec-11f0-b747-eadd75232b79/coh_vv.tif>}